# Appendix 5: Testing the new likelihood interface

Johann Brehmer, Felix Kling, Irina Espejo, and Kyle Cranmer 2018-2019

In [1]:
import logging
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline
from scipy.optimize import minimize
from scipy.stats import chi2, poisson

# MadMiner output
logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)

# Output of all other modules (e.g. matplotlib)"
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

from madminer import sampling
from madminer.sampling import SampleAugmenter
from madminer.ml import ParameterizedRatioEstimator
from madminer.likelihood import NeuralLikelihood, fix_params
from madminer.utils.various import less_logging

## Sampling and training

In [ ]:
sampler = SampleAugmenter("data/lhe_data_systematics.h5")

_ = sampler.sample_train_ratio(
    theta0=sampling.random_morphing_points(
        1000, [("gaussian", 0.0, 15.0), ("gaussian", 0.0, 15.0)]
    ),
    theta1=sampling.benchmark("sm"),
    nu0=sampling.iid_nuisance_parameters(),
    nu1=sampling.nominal_nuisance_parameters(),
    n_samples=10000,
    folder="./data/samples",
    filename="train_ratio_systematics",
    sample_only_from_closest_benchmark=True,
    return_individual_n_effective=True,
)

In [5]:
estimator = ParameterizedRatioEstimator(n_hidden=(100,), activation="tanh")

estimator.train(
    method="alices",
    theta="data/samples/theta0_train_ratio_systematics.npy",
    x="data/samples/x_train_ratio_systematics.npy",
    y="data/samples/y_train_ratio_systematics.npy",
    r_xz="data/samples/r_xz_train_ratio_systematics.npy",
    t_xz="data/samples/t_xz_train_ratio_systematics.npy",
    alpha=1.0,
    n_epochs=20,
)

estimator.save("models/alices_systematics")

16:29 madminer.ml          INFO    Starting training
16:29 madminer.ml          INFO      Method:                 alices
16:29 madminer.ml          INFO      alpha:                  1.0
16:29 madminer.ml          INFO      Batch size:             128
16:29 madminer.ml          INFO      Optimizer:              amsgrad
16:29 madminer.ml          INFO      Epochs:                 20
16:29 madminer.ml          INFO      Learning rate:          0.001 initially, decaying to 0.0001
16:29 madminer.ml          INFO      Validation split:       0.25
16:29 madminer.ml          INFO      Early stopping:         True
16:29 madminer.ml          INFO      Scale inputs:           True
16:29 madminer.ml          INFO      Scale parameters:       True
16:29 madminer.ml          INFO      Shuffle labels          False
16:29 madminer.ml          INFO      Samples:                all
16:29 madminer.ml          INFO    Loading training data
16:29 madminer.utils.vario INFO      Loading data/samples/theta0_t

16:30 madminer.utils.ml.tr INFO                        early stopping:   0.00h
16:30 madminer.utils.ml.tr INFO                          report epoch:   0.00h
16:30 madminer.ml          INFO    Saving model to models/alices_systematics


## Create likelihood function

In [2]:
likelihood = NeuralLikelihood("data/lhe_data_systematics.h5")

nll = likelihood.create_expected_negative_log_likelihood(
    model_file="models/alices_systematics",
    n_asimov=1000,
    include_xsec=True,
    theta_true=np.array([0.0, 0.0]),
    nu_true=np.array([0.0, 0.0, 0.0]),
    xsec_mode="interpol",
)

16:10 madminer.analysis.da INFO    Loading data from data/lhe_data_systematics.h5
16:10 madminer.analysis.da INFO    Found 2 parameters
16:10 madminer.analysis.da INFO    Found 3 nuisance parameters
16:10 madminer.analysis.da INFO    Found 10 benchmarks, of which 6 physical
16:10 madminer.analysis.da INFO    Found 3 observables
16:10 madminer.analysis.da INFO    Found 57687 events
16:10 madminer.analysis.da INFO      9858 signal events sampled from benchmark sm
16:10 madminer.analysis.da INFO      47829 background events
16:10 madminer.analysis.da INFO    Found morphing setup with 6 components
16:10 madminer.analysis.da INFO    Found nuisance morphing setup
16:10 madminer.ml.base     INFO    Loading model from models/alices_systematics


## Unconstrained fit (overall best theta / nu)

In [3]:
result = minimize(
    nll,
    x0=np.array([0.1, -0.1, 0.1, -0.1, 0.1]),
    method="L-BFGS-B",
)
best_fit = result.x
best_fit

array([ 3.00695662, -0.12703062, -0.51023258,  6.11995267,  3.31061421])

## Cross-check: explicit xsec calculation (no interpolation)

In [4]:
likelihood2 = NeuralLikelihood("data/lhe_data_systematics.h5")

nll2 = likelihood2.create_expected_negative_log_likelihood(
    model_file="models/alices_systematics",
    n_asimov=1000,
    include_xsec=True,
    theta_true=np.array([0.0, 0.0]),
    nu_true=np.array([0.0, 0.0, 0.0]),
    xsec_mode="blablubb",
)

result2 = minimize(
    nll2,
    x0=np.array([0.1, -0.1, 0.1, -0.1, 0.1]),
    method="L-BFGS-B",
)
best_fit2 = result2.x
best_fit2

16:10 madminer.analysis.da INFO    Loading data from data/lhe_data_systematics.h5
16:10 madminer.analysis.da INFO    Found 2 parameters
16:10 madminer.analysis.da INFO    Found 3 nuisance parameters
16:10 madminer.analysis.da INFO    Found 10 benchmarks, of which 6 physical
16:10 madminer.analysis.da INFO    Found 3 observables
16:10 madminer.analysis.da INFO    Found 57687 events
16:10 madminer.analysis.da INFO      9858 signal events sampled from benchmark sm
16:10 madminer.analysis.da INFO      47829 background events
16:10 madminer.analysis.da INFO    Found morphing setup with 6 components
16:10 madminer.analysis.da INFO    Found nuisance morphing setup
16:10 madminer.ml.base     INFO    Loading model from models/alices_systematics


array([ 3.00695662, -0.12703062, -0.51023258,  6.11995267,  3.31061421])

## Fixing theta = [0., 0.], fitting nu

In [5]:
constrained_nll = fix_params(nll, np.array([0.0, 0.0]))

result = minimize(
    constrained_nll,
    x0=np.array([0.1, -0.1, 0.1]),
    method="L-BFGS-B",
)
constrained_best_fit = result.x
constrained_best_fit

array([ 0.1       , -0.09999823,  0.09999992])

## Calculating the profile log likelihood ratio (q) and asymptotic p-value

In [6]:
q = 2.0 * (constrained_nll(constrained_best_fit) - nll(best_fit))
q

array([2342.23339313])

In [7]:
dof = 2
p_value = chi2.sf(x=q, df=dof)
p_value

array([0.])

## We've excluded the SM at super high significance :D